In [2]:
import pandas as pd
import sqlite3
DB_PATH = "alchemy.db"
EFFECT_SQL_QUERY = """
SELECT i.code, e.description, et."type",et.value FROM ingredients AS i
join properties props on i.id = props.ingredient_id 
join effects e on e.id =props.effect_id
left join effects_types et on e.id =et.effect_id
where i.code =? and props.ingredient_order =?
"""
SELECT_BY_EFFECT_TYPE_CODE_NOT_IN = """
SELECT i.code, props.ingredient_order, e.description, et."type",et.value FROM ingredients AS i
join properties props on i.id = props.ingredient_id 
join effects e on e.id =props.effect_id
left join effects_types et on e.id =et.effect_id
where i.code not in {ingredients} and et."type" in {effects_types}
"""
SELECT_ALL_EFFECTS="""
SELECT i.code, props.ingredient_order , e.description, et."type",et.value FROM ingredients AS i
join properties props on i.id = props.ingredient_id 
join effects e on e.id =props.effect_id
left join effects_types et on e.id =et.effect_id
where et."type" is not Null
"""
def get_by_code_order(ingredient_code,ingredient_order,cursor):
    cursor.execute(EFFECT_SQL_QUERY,(ingredient_code,ingredient_order))
    found_effects = cursor.fetchall()
    found_effects_df =pd.DataFrame(data=found_effects,columns=["code","description","effect_type","effect_value"])
    return found_effects_df
def evaluate_effects(other_effects_df,grouped_effects):
    score = 0
    other_effects_len = other_effects_df.shape[0]
    if other_effects_len > 4:
        score -= 1000
    else:
        score -= 100 * other_effects_len
        
    for effect_type,effect_df in grouped_effects:
        effect_value = effect_df["effect_value"].sum()
        if effect_value>0:
            score+=10
        else:
            score-=100
    return score
def get_all_effects(ingredients,ingredients_effects_nums,cursor):
    all_effects_df=[]
    for ingredient_code,ingredient_effect_i in zip(ingredients,ingredients_effects_nums):
        all_effects_df.append(get_by_code_order(ingredient_code=ingredient_code,ingredient_order=ingredient_effect_i,cursor=cursor))
    for ingredient_code in ingredients:
        all_effects_df.append(get_by_code_order(ingredient_code=ingredient_code,ingredient_order=0,cursor=cursor))
    all_effects_df = pd.concat(all_effects_df)
    return all_effects_df
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()


In [3]:
formula = [
    # "MER1",
    "YN2","TD1"]

ingredients = [x[:-1] for x in formula]
ingredients_effects_nums = [int(x[-1]) for x in formula]


In [4]:
    
all_effects_df = get_all_effects(ingredients,ingredients_effects_nums,cursor)
other_effects_df = all_effects_df[all_effects_df["effect_type"].isna()]
grouped_effects = list(all_effects_df.groupby("effect_type"))

In [5]:
current_effect_groups = [x for x in all_effects_df['effect_type'].unique() if x is not None]
if len(current_effect_groups)==1:
    current_effect_groups = (current_effect_groups[0],current_effect_groups[0])

In [6]:
query = SELECT_BY_EFFECT_TYPE_CODE_NOT_IN.format(ingredients=tuple(ingredients),effects_types=current_effect_groups)
cursor.execute(query)
cursor.fetchall()

[('FS', 3, 'средний яд', 'яд', -2),
 ('FR', 1, 'смертельный яд', 'яд', -4),
 ('QCH',
  3,
  'наносит повреждения магическим созданиям (работает как слабый яд для существ)',
  'яд',
  -1),
 ('EP', 3, 'сильный яд', 'яд', -3),
 ('SP', 2, 'средний яд', 'яд', -2),
 ('VZ', 3, 'сильное противоядие', 'яд', 3),
 ('CN', 0, 'средний яд', 'яд', -2),
 ('CN', 3, 'слабый яд', 'яд', -1),
 ('GSI', 2, 'противоядие против средних ядов', 'яд', 2),
 ('HV', 1, 'среднее противоядие', 'яд', 2),
 ('SC', 0, 'слабый яд', 'яд', -1),
 ('SZ', 2, 'среднее противоядие', 'яд', 2),
 ('RK', 1, 'способно спасти от смертельных ядов, противоядие', 'яд', 4),
 ('OM', 0, 'средний яд (открывает кровотечение)', 'яд', -2),
 ('BN', 2, 'слабый яд (понижает энергию)', 'яд', -1),
 ('BA',
  0,
  'смертельный яд для перевёртышей (обладатели таланта "превращение")',
  'яд',
  -4),
 ('ST', 3, 'слабое противоядие', 'яд', 1),
 ('GRO',
  0,
  'ядовито для женщин, при контакте приводик к сну без сноведений (правила магической комы). у мужчи

In [7]:
cursor.execute(SELECT_ALL_EFFECTS)
all_ingredients_effects = cursor.fetchall()
all_ingredients_effects = pd.DataFrame(data=all_ingredients_effects, columns=["code","ingredient_order","description","effect_type","effect_value"])

In [13]:
effect_type = "яд"
all_ingredients_effects[all_ingredients_effects["effect_type"]==effect_type]

,code,ingredient_order,description,effect_type,effect_value
0,FS,3,средний яд,яд,-2
2,FR,1,смертельный яд,яд,-4
10,QCH,3,наносит повреждения магическим созданиям (рабо...,яд,-1
14,EP,3,сильный яд,яд,-3
15,YN,0,смертельный яд,яд,-4
18,SP,2,средний яд,яд,-2
24,VZ,3,сильное противоядие,яд,3
27,CN,0,средний яд,яд,-2
28,CN,3,слабый яд,яд,-1
34,GSI,2,противоядие против средних ядов,яд,2
